#Introdução

Este notebook é dedicado à feature engineering da tabela Installments Payments . Neste contexto, estarei explorando e transformando os dados da tabela para otimizar o desempenho dos modelos de machine learning que serão aplicados posteriormente. Feature engineering é uma etapa crucial no processo de preparação de dados, onde buscamos extrair informações relevantes e criar novas variáveis (ou features) que possam melhor capturar os padrões subjacentes aos nossos dados.

No contexto específico do nosso negócio de concessão de crédito, a feature engineering desempenha um papel fundamental. A capacidade de prever com precisão a probabilidade de inadimplência ou de pagamento pontual é crucial para a saúde financeira da instituição. Nesse sentido, a feature engineering permite não apenas melhorar a eficácia dos modelos de machine learning em realizar essas previsões, mas também oferece uma oportunidade de compreender melhor o comportamento dos clientes e os fatores que influenciam sua capacidade de pagamento.

O objetivo principal da feature engineering é, portanto, melhorar a capacidade dos modelos de machine learning em aprender com os dados, aumentando assim sua eficácia na realização de previsões ou classificações. Isso é alcançado através da seleção, transformação e criação de features que possam fornecer insights mais significativos e representativos para o problema em questão. Ao final deste processo, que será feito em todas as tabelas, esperamos obter um conjunto de dados mais refinado e adequado para alimentar nossos modelos de machine learning, resultando em predições mais precisas e confiáveis, o que é essencial para a tomada de decisões assertivas no âmbito da concessão de crédito.

##Configuração do Ambiente para Utilização do Spark
Configuração do ambiente e ferramentas essenciais que foram instaladas para o início do projeto.

Instalação do Apache Spark: O Apache Spark, uma poderosa ferramenta para processamento distribuído de dados em grande escala, foi instalado no ambiente de desenvolvimento. Esta instalação permitirá a utilização de todas as funcionalidades oferecidas pelo Spark para análise e manipulação de dados.

Configuração do Ambiente Python: Para interagir de forma eficiente com o Spark utilizando a linguagem Python, foram realizadas configurações específicas do ambiente Python. Isso inclui a instalação do pacote PySpark, uma biblioteca Python que fornece uma API para interagir com o Spark.

Criação da Sessão Spark: Uma sessão Spark foi criada utilizando a classe SparkSession. Essa sessão é fundamental para estabelecer uma conexão com o ambiente Spark e executar operações de processamento de dados distribuídas. Através dessa sessão, será possível acessar todos os recursos e funcionalidades do Spark para realizar análises e manipulações nos dados do projeto.

In [6]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Minha Primeira Aplicação no Pyspark") \
        .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

## Leitura dos Dados

In [7]:
df_Installments = spark.read.csv ('/content/drive/MyDrive/Colab Notebooks/AnaliseCredito/installments_payments.csv', header=True, inferSchema=True)
df_Installments.createOrReplaceTempView("df_Installments")

In [3]:
df_Installments.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- NUM_INSTALMENT_VERSION: double (nullable = true)
 |-- NUM_INSTALMENT_NUMBER: integer (nullable = true)
 |-- DAYS_INSTALMENT: double (nullable = true)
 |-- DAYS_ENTRY_PAYMENT: double (nullable = true)
 |-- AMT_INSTALMENT: double (nullable = true)
 |-- AMT_PAYMENT: double (nullable = true)



In [4]:
df_Installments.show()

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|   1054186|    161674|                   1.0|                    6|        -1180.0|           -1187.0|       6948.36|    6948.36|
|   1330831|    151639|                   0.0|                   34|        -2156.0|           -2156.0|      1716.525|   1716.525|
|   2085231|    193053|                   2.0|                    1|          -63.0|             -63.0|       25425.0|    25425.0|
|   2452527|    199697|                   1.0|                    3|        -2418.0|           -2426.0|      24350.13|   24350.13|
|   2714724|    167756|                   1.0|                    2|        -1383.0

## Criação de Flags para Visão Temporal dos Dados

Criação de flags para representar o comportamento do cliente ao longo de diferentes períodos de tempo em relação aos seus pagamentos.

In [8]:
df_flag_01 = spark.sql("""
SELECT *,
  CASE
    WHEN DAYS_INSTALMENT >= -90 THEN 1
    ELSE 0
    END AS ULT_3_MESES_INSTALMENT,
  CASE
    WHEN DAYS_INSTALMENT >= -180 THEN 1
    ELSE 0
    END AS ULT_6_MESES_INSTALMENT,
  CASE
    WHEN DAYS_INSTALMENT >= -270 THEN 1
    ELSE 0
    END AS ULT_9_MESES_INSTALMENT,
  CASE
    WHEN DAYS_INSTALMENT >= -360 THEN 1
    ELSE 0
    END AS ULT_12_MESES_INSTALMENT,
  CASE
    WHEN DAYS_INSTALMENT >= -540 THEN 1
    ELSE 0
    END AS ULT_18_MESES_INSTALMENT
FROM
    df_Installments
ORDER BY
    SK_ID_PREV
""")
df_flag_01.show()

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|ULT_3_MESES_INSTALMENT|ULT_6_MESES_INSTALMENT|ULT_9_MESES_INSTALMENT|ULT_12_MESES_INSTALMENT|ULT_18_MESES_INSTALMENT|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+
|   1000005|    176456|                   1.0|                    2|        -1658.0|           -1664.0|     14713.605|  14713.605|                     0|                     0|                     0|                      0|                      0|
|   1000

##Criação da Variável de Razão de Pagamento

Criação da variável relação entre o valor pago e o valor da parcela nos empréstimos ou financiamentos dos clientes.

In [9]:
df_flag_01.createOrReplaceTempView("df_flag_01")
df_razao_pagamento = spark.sql("""
  SELECT *,
    ROUND(AMT_PAYMENT / AMT_INSTALMENT,2) AS RAZAO_PAG_E_VL_PARCELA
  FROM
    df_flag_01;
""")
df_razao_pagamento.show()

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|ULT_3_MESES_INSTALMENT|ULT_6_MESES_INSTALMENT|ULT_9_MESES_INSTALMENT|ULT_12_MESES_INSTALMENT|ULT_18_MESES_INSTALMENT|RAZAO_PAG_E_VL_PARCELA|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+
|   1000005|    176456|                   1.0|                    2|        -1658.0|           -1664.0|     14713.605|  14713.605|                     0|                     0|           

##Criação da Variável de Atraso de Pagamento

Criação da variável atrasos nos pagamentos dos clientes.

In [10]:
df_razao_pagamento.createOrReplaceTempView("df_razao_pagamento")
df_atraso_pagamento = spark.sql("""
  SELECT *,
    DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT AS ATRASO_PAGAMENTO
  FROM
   df_razao_pagamento
""")
df_atraso_pagamento.show()

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|ULT_3_MESES_INSTALMENT|ULT_6_MESES_INSTALMENT|ULT_9_MESES_INSTALMENT|ULT_12_MESES_INSTALMENT|ULT_18_MESES_INSTALMENT|RAZAO_PAG_E_VL_PARCELA|ATRASO_PAGAMENTO|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------+
|   1000005|    176456|                   1.0|                    2|        -1658.0|           -1664.0|     14713.605|  14713.605|      

 ## Cálculo da Quantidade de Pagamento Recebido pelo Cliente

 Diferença entre o valor pago e o valor da parcela esperada para cada transação.

In [11]:
df_atraso_pagamento.createOrReplaceTempView("df_atraso_pagamento")
df_qtde_pag_recebido = spark.sql("""
    SELECT *,
        AMT_PAYMENT - AMT_INSTALMENT AS QUANTIDADE_PAGAMENTO_RECEBIDO
    FROM
        df_atraso_pagamento
""")
df_qtde_pag_recebido.show()


+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------+-----------------------------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|ULT_3_MESES_INSTALMENT|ULT_6_MESES_INSTALMENT|ULT_9_MESES_INSTALMENT|ULT_12_MESES_INSTALMENT|ULT_18_MESES_INSTALMENT|RAZAO_PAG_E_VL_PARCELA|ATRASO_PAGAMENTO|QUANTIDADE_PAGAMENTO_RECEBIDO|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------+-----------------------------+
|   1000005|    176456|                   1.0|

## Cálculo do Número Total de Pagamentos por Cliente

Agregação do número total de pagamentos feitos por cada cliente.

In [12]:
df_qtde_pag_recebido.createOrReplaceTempView("df_qtde_pag_recebido")
df_numero_tot_pag = spark.sql("""
SELECT
    SK_ID_PREV,
    COUNT(*) AS NUMERO_TOTAL_PAGAMENTOS
FROM
   df_qtde_pag_recebido
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
""")
df_numero_tot_pag.show()

+----------+-----------------------+
|SK_ID_PREV|NUMERO_TOTAL_PAGAMENTOS|
+----------+-----------------------+
|   1000005|                      1|
|   1000011|                      1|
|   1000016|                      1|
|   1000019|                      2|
|   1000025|                      2|
|   1000031|                      4|
|   1000032|                      1|
|   1000034|                      1|
|   1000042|                      1|
|   1000045|                      2|
|   1000048|                      3|
|   1000049|                      1|
|   1000050|                      1|
|   1000052|                      2|
|   1000054|                      1|
|   1000057|                      3|
|   1000058|                      1|
|   1000060|                      1|
|   1000063|                      2|
|   1000073|                      2|
+----------+-----------------------+
only showing top 20 rows



##Criação de Variáveis Agregadas de Pagamento

Agregando informações sobre os valores de pagamento ao longo de diferentes períodos de tempo.

In [13]:
df_qtde_pag_recebido.createOrReplaceTempView("df_qtde_pag_recebido")
df_temp = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MED_AMT_INSTALMENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MIN_AMT_INSTALMENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MAX_AMT_INSTALMENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_SUM_AMT_INSTALMENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MED_AMT_INSTALMENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MIN_AMT_INSTALMENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MAX_AMT_INSTALMENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_SUM_AMT_INSTALMENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MED_AMT_INSTALMENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MIN_AMT_INSTALMENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MAX_AMT_INSTALMENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_SUM_AMT_INSTALMENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MED_AMT_INSTALMENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MIN_AMT_INSTALMENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MAX_AMT_INSTALMENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_SUM_AMT_INSTALMENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MED_AMT_INSTALMENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MIN_AMT_INSTALMENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_MAX_AMT_INSTALMENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_INSTALMENT ELSE NULL END) AS VL_SUM_AMT_INSTALMENT_ULT_18_MS
FROM
    df_qtde_pag_recebido
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp.show()

+----------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+
|SK_ID_PREV|VL_MED_AMT_INSTALMENT_ULT_3_MS|VL_MIN_AMT_INSTALMENT_ULT_3_MS|VL_MAX_AMT_INSTALMENT_ULT_3_MS|VL_SUM_AMT_INSTALMENT_ULT_3_MS|VL_MED_AMT_INSTALMENT_ULT_6_MS|VL_MIN_AMT_INSTALMENT_ULT_6_MS|VL_MAX_AMT_INSTALMENT_ULT_6_MS|VL_SUM_AMT_INSTALMENT_ULT_6_MS|VL_MED_AMT_INSTALMENT_ULT_9_MS|VL_MIN_AMT_INSTALMENT_ULT_9_MS|VL_MAX_AMT_INSTALMENT_ULT_9_MS|VL_SUM

##Criação de Variáveis Agregadas de Pagamento Recente

Agregando informações sobre os pagamentos dos clientes, focando agora nos valores de pagamento recentes.

In [14]:
df_temp_01 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES_INSTALMENT = 1 THEN AMT_PAYMENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_ULT_18_MS
FROM
    df_qtde_pag_recebido
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_01.show()

+----------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+
|SK_ID_PREV|VL_MED_AMT_PAYMENT_ULT_3_MS|VL_MIN_AMT_PAYMENT_ULT_3_MS|VL_MAX_AMT_PAYMENT_ULT_3_MS|VL_SUM_AMT_PAYMENT_ULT_3_MS|VL_MED_AMT_PAYMENT_ULT_6_MS|VL_MIN_AMT_PAYMENT_ULT_6_MS|VL_MAX_AMT_PAYMENT_ULT_6_MS|VL_SUM_AMT_PAYMENT_ULT_6_MS|VL_MED_AMT_PAYMENT_ULT_9_MS|VL_MIN_AMT_PAYMENT_ULT_9_MS|VL_MAX_AMT_PAYMENT_ULT_9_MS|VL_SUM_AMT_PAYMENT_ULT_9_MS|VL_MED_AMT_PAYMENT_ULT_12_MS|VL_MIN_AMT_PAYMENT_ULT_12_MS|VL_MAX_AMT_PA

## Realização de Joins entre DataFrames
O DataFrame resultante dessas operações de junção contém todas as informações agregadas e combinadas das variáveis criadas anteriormente.

In [19]:
# Realizar o join
df_installments_final = df_qtde_pag_recebido.join(df_numero_tot_pag, on='SK_ID_PREV', how='inner')
df_installments_final = df_installments_final.join(df_temp_01, on='SK_ID_PREV', how='inner')


# Exibir o DataFrame resultante
df_installments_final.show()


+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------+-----------------------------+-----------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_E

## Agregação de Variáveis na Visão Cliente
Realização de uma sumarização dos dados na perspectiva do cliente.

In [20]:
from pyspark.sql.functions import first

# Lista de todas as colunas, exceto a coluna SK_ID_PREV
colunas_para_agregar = [col for col in df_installments_final.columns if col != "SK_ID_PREV"]

# Agregue as variáveis por SK_ID_PREV
df_installments_final_01 = df_installments_final.groupBy("SK_ID_PREV").agg(
    *[first(col).alias(col) for col in colunas_para_agregar]
)

# Mostre o resultado
df_installments_final_01.show()

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------+-----------------------------+-----------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_E

## Salvar a tabela Sumarizada

In [21]:
df_installments_final_01 = df_installments_final_01.repartition(1)
df_installments_final_01.write.mode("overwrite").csv("installments_payments_agg.csv",header=True)